In [1]:
from langchain_openai import ChatOpenAI

In [4]:
chat = ChatOpenAI()

## Messages
聊天模型的接口是围绕Messages类设计的。目前LangChain支持AIMessage、HumanMessage，SystemMessage，FunctionMessage和ChatMessage。ChatMessage接收一个任意的用户参数。

## LCEL
Chat Model 实现了LangChain的Runnable interface接口，因此Chat Model支持invoke、ainvoke等接口。

Chat Model接收List\[BaseMessage\]作为输入。

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

In [6]:
messages = [
    SystemMessage(content="You're a helpful assistant"),
    HumanMessage(content="What is the purpose of model regularization?")
]

In [7]:
chat.invoke(messages)

AIMessage(content="The purpose of model regularization is to prevent overfitting in machine learning models. Overfitting occurs when a model learns the noise and fluctuations in the training data rather than the underlying patterns, which can lead to poor performance on unseen data. Regularization techniques introduce constraints or penalties to the model's optimization process to discourage the model from fitting the training data too closely. This helps improve the generalization ability of the model and make it more robust to unseen data, ultimately leading to better performance in real-world applications.", response_metadata={'finish_reason': 'stop', 'logprobs': None})

## Function calling

In [11]:
import getpass
import os
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

In [10]:
class Multiply(BaseModel):
    """Multiply two integers together."""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [13]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools([Multiply])
llm_with_tools.invoke("what's 3*12")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Z5hv8mGB9jfKm4aSCWxd23yI', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None})

In [14]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

tool_chain = llm_with_tools | JsonOutputToolsParser()
tool_chain.invoke("what's 3*12")  # 这个过程是怎么发生的？

[{'type': 'Multiply', 'args': {'a': 3, 'b': 12}}]

In [15]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

tool_chain = llm_with_tools | PydanticToolsParser(tools=[Multiply])
tool_chain.invoke("what's 3*12")

[Multiply(a=3, b=12)]

In [16]:
llm_with_multiply = llm.bind_tools([Multiply], tool_choice="Multiply")
llm_with_multiply.invoke(
    "make up some numbers if you really want but I'm not forcing you"
)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Jt69CXmLUBtz1ihlIQ9gRa7c', 'function': {'arguments': '{"a":5,"b":10}', 'name': 'Multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'stop', 'logprobs': None})

## Defining functions schemas
我们可以自定义工具，然后使用langchain将我们的工具转换成OpenAI的格式。

In [17]:
import json

from langchain_core.utils.function_calling import convert_to_openai_tool

In [20]:
def multiply(a:int, b:int) -> int:
    """Multiply two integers together.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

print(json.dumps(convert_to_openai_tool(multiply), indent=2))

{
  "type": "function",
  "function": {
    "name": "multiply",
    "description": "Multiply two integers together.",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "type": "integer",
          "description": "First integer"
        },
        "b": {
          "type": "integer",
          "description": "Second integer"
        }
      },
      "required": [
        "a",
        "b"
      ]
    }
  }
}


In [22]:
class multiply1(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

print(json.dumps(convert_to_openai_tool(multiply1), indent=2))

{
  "type": "function",
  "function": {
    "name": "multiply1",
    "description": "Multiply two integers together.",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "description": "First integer",
          "type": "integer"
        },
        "b": {
          "description": "Second integer",
          "type": "integer"
        }
      },
      "required": [
        "a",
        "b"
      ]
    }
  }
}


## LangChain Tool

In [23]:
from typing import Any, Type

from langchain_core.tools import BaseTool

class MultiplySchema(BaseModel):
    """Multiply tool schema"""
    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class Multiply(BaseTool):
    args_schema: Type[BaseModel] = MultiplySchema
    name: str = "multiply"
    description: str = "Multiply two integers together."

    def _run(self, a: int, b: int, **kwargs: any) -> Any:
        return a*b

print(json.dumps(convert_to_openai_tool(Multiply()), indent=2))

{
  "type": "function",
  "function": {
    "name": "multiply",
    "description": "Multiply two integers together.",
    "parameters": {
      "type": "object",
      "properties": {
        "a": {
          "description": "First integer",
          "type": "integer"
        },
        "b": {
          "description": "Second integer",
          "type": "integer"
        }
      },
      "required": [
        "a",
        "b"
      ]
    }
  }
}
